In [ ]:
import pandas as pd
import torch
import numpy as np
import csv
import random
import os

import sys

from torch.autograd import Variable
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
objList = {0: "cherry", 1:"eraser", 2:"lemon", 3:"lemonPiece", 4:"lightbulb", 5:"pear", 6:"pinecone", 7:"strawberry", 8:"tapeDispenser", 9:"yuanbao"}

class objClsf(Dataset):
    def __init__(self, data):
        super().__init__()
        self.labelDict = {"cherry":0, "eraser":1, "lemon":2, "lemonPiece":3, "lightbulb":4, "pear":5, "pinecone":6, "strawberry":7, "tapeDispenser":8, "yuanbao":9}
        random.shuffle(data)
        self.data = data

    def __getitem__(self, index):
        d = self.data[index][0]
        d = torch.tensor(d, dtype=torch.float32) /229.0
        return d, self.labelDict[self.data[index][1]]
        
    def __len__(self):
        return len(self.data)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.sq = nn.Sequential()
        self.fc1 = nn.Linear(128,32)
        self.fc2 = nn.Linear(32,10)
        self.sm = nn.Softmax(dim=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.fc1((x)))
        out = self.sm(self.fc2(out))
        return out
    
def Train(epochs, trainLoader, model, device, learningRate):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
    model.to(device)
    correct = 0
    total = 0
    for e in range(epochs):
        runningloss = 0
        correct = 0
        total = 0
        for i, (data, label) in enumerate(trainLoader):
            optimizer.zero_grad()
            data = data.to(device)
            label = label.to(device)
            out = model(data)
            _, pre = torch.max(out, 1)
            total += label.size(0)
            correct += (pre == label).sum().item()
            loss = criterion(out, label)
            loss.backward()
            optimizer.step()
            runningloss += loss.data
            if (i+1)%10 == 0:
                print('epoch: {}, batch: {}, loss:{}'.format(e+1, i+1, runningloss/(i+1)))
        print('Accuracy: {}'.format(correct/total))
    torch.save(model, 'objC.pth')

def Test(testLoader, model, device):
    model.to(device)
    torch.no_grad()
    correct = 0
    total = 0
    preL = []
    labelL = []
    for (data, label) in testLoader:   
        data = data.to(device)
        label = label.to(device)
        out = model(data)
        _, pre = torch.max(out, 1)
        total += label.size(0)
        correct += (pre == label).sum().item()
        preL += list(pre.cpu().numpy())
        labelL += list(label.cpu().numpy())
    print('Accuracy: {}'.format(correct/total))
    return preL, labelL

In [ ]:

testList = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
testDataset =  objClsf(testList)    
test_loader = DataLoader(testDataset, batch_size=16, shuffle= True)
model = torch.load('objC_10_objects.pth')
res = Test(test_loader, model, device)